In [ ]:
# Gerekli kütüphanelerin import edilmesi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, RandomizedSearchCV # DEĞİŞİKLİK: GridSearchCV yerine RandomizedSearchCV import edildi
from sklearn.preprocessing import RobustScaler # DEĞİŞİKLİK: MinMaxScaler yerine RobustScaler import edildi
from sklearn.impute import SimpleImputer
import xgboost as xgb
import warnings
import shap
# Uyarıları filtrele
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:



# 1. Veri Yükleme
dftrain = pd.read_csv("/kaggle/input/playground-series-s5e4/train.csv")
dftest = pd.read_csv("/kaggle/input/playground-series-s5e4/test.csv")

# Orijinal test ID'lerini daha sonra kullanmak üzere sakla
test_ids = dftest['id']


In [ ]:

# 2. Ön İşleme - Kategorik Değişkenlerin Dönüştürülmesi (One-Hot Encoding)
# Kategorik sütunları belirle
objeler_train = dftrain.select_dtypes(include=['object']).columns
objeler_test = dftest.select_dtypes(include=['object']).columns

# One-hot encoding uygula
dftrain_dummies = pd.get_dummies(dftrain, columns=objeler_train, drop_first=True, dtype=int)
dftest_dummies = pd.get_dummies(dftest, columns=objeler_test, drop_first=True, dtype=int)


In [ ]:


# 3. Sütunları Hizalama
# 'id' sütununu ve hedef değişkeni ayır
dfy = dftrain_dummies["Listening_Time_minutes"]
dfx = dftrain_dummies.drop(columns=["id", "Listening_Time_minutes"])
dftest_processed = dftest_dummies.drop(columns=["id"])

# Eğitim ve test setlerindeki sütunları eşleştir
train_cols = dfx.columns
test_cols = dftest_processed.columns

missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    dftest_processed[c] = 0

missing_in_train = set(test_cols) - set(train_cols)
for c in missing_in_train:
    dfx[c] = 0

# Sütun sıralamasını aynı yap
dftest_processed = dftest_processed[train_cols]

print("✅ Eğitim ve test seti sütunları hizalandı.")
print("Eğitim seti boyutu:", dfx.shape)
print("Test seti boyutu:", dftest_processed.shape)



In [ ]:

# 4. Eksik Değerlerin Doldurulması (Imputation)
imputer = SimpleImputer(strategy='median')

# İmputeri SADECE eğitim verisiyle eğit (fit) ve ardından her iki seti de dönüştür (transform)
dfx_imputed = imputer.fit_transform(dfx)
dftest_imputed = imputer.transform(dftest_processed)

# Dönüştürme sonrası numpy array'lerini tekrar DataFrame'e çeviriyoruz
dfx_imputed = pd.DataFrame(dfx_imputed, columns=train_cols)
dftest_imputed = pd.DataFrame(dftest_imputed, columns=train_cols)

print("\n✅ Eksik değerler 'median' stratejisi ile dolduruldu.")



In [ ]:

# 5. Veri Ölçeklendirme (Scaling)
# DEĞİŞİKLİK: MinMaxScaler yerine RobustScaler kullanıldı.
# RobustScaler, aykırı değerlerden (outliers) daha az etkilenir.
scaler = RobustScaler()
dftrain_scaled = scaler.fit_transform(dfx_imputed)
dftest_scaled = scaler.transform(dftest_imputed)

print("\n✅ Veriler RobustScaler ile ölçeklendirildi.")



In [ ]:

# 6. Model Eğitimi ve Hiperparametre Optimizasyonu
cv = KFold(n_splits=5, shuffle=True, random_state=42)

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    device='cuda' # GPU kullanmıyorsanız bu satırı silin veya 'cpu' olarak değiştirin
)

# DEĞİŞİKLİK: RandomizedSearchCV için daha geniş bir parametre dağılımı tanımlandı.
# Bu dağılımdan rastgele kombinasyonlar denenecektir.
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.15],
    'max_depth': [5, 7, 9, 11, 13],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.01, 0.1]
}

# DEĞİŞİKLİK: GridSearchCV yerine RandomizedSearchCV kullanıldı.
# n_iter: Denenecek rastgele parametre kombinasyonu sayısı.
# Bu, tüm kombinasyonları denemekten daha hızlıdır ve genellikle çok iyi sonuçlar bulur.
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=50,  # 50 farklı kombinasyon dene
    scoring='r2',
    cv=cv,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# Modeli eğit
print("\n⏳ RandomizedSearchCV ile model eğitimi başladı...")
random_search.fit(dftrain_scaled, dfy)

# Sonuçlar
print("\n✅ Model eğitimi tamamlandı.")
print("En iyi parametreler:", random_search.best_params_)
print("En iyi R² skoru:", random_search.best_score_)

# En iyi modeli seç
best_model = random_search.best_estimator_



In [ ]:

# 7. Tahmin Yapma
print("\n⏳ Test verisi üzerinde tahmin yapılıyor...")
predictions = best_model.predict(dftest_scaled)
print("✅ Tahminler yapıldı.")



In [ ]:

# 8. Teslimat Dosyasını Oluşturma
df_submit = pd.DataFrame({
    "id": test_ids,
    "Listening_Time_minutes": predictions
})

# Negatif tahminleri 0'a yuvarla (dinleme süresi negatif olamaz)
df_submit['Listening_Time_minutes'] = df_submit['Listening_Time_minutes'].clip(lower=0)

df_submit.to_csv("submission.csv", index=False)
print("\n🚀 submission.csv dosyası başarıyla oluşturuldu!")



In [ ]:

# 9. (Opsiyonel) SHAP ile Model Açıklanabilirliği
# Bellek sorunlarını önlemek için SHAP analizini verinin bir alt kümesi üzerinde yapıyoruz.
X_sample = pd.DataFrame(dftrain_scaled, columns=train_cols).sample(5000, random_state=42)

print("\n⏳ SHAP değerleri hesaplanıyor (bir örneklem üzerinde)...")
explainer = shap.Explainer(best_model) # GPU kullanılıyorsa bazen `X_sample` burada da gerekebilir.
shap_values = explainer(X_sample)

# Özet grafiğini çizdir
print("✅ SHAP özeti:")
plt.title("SHAP Feature Importance Summary")
shap.summary_plot(shap_values, X_sample, show=False)
plt.show()